In [1]:
from peewee import *

db = SqliteDatabase('people.db')

class Person(Model):
    name = CharField()
    birthday = DateField()

    class Meta:
        database = db # This model uses the "people.db" database.

In [2]:
class Pet(Model):
    owner = ForeignKeyField(Person, backref='pets')
    name = CharField()
    animal_type = CharField()

    class Meta:
        database = db # this model uses the "people.db" database

In [3]:
db.connect()

OperationalError: Connection already opened.

In [5]:
db.create_tables([Person, Pet])

In [6]:
from datetime import date
uncle_bob = Person(name='Bob', birthday=date(1960, 1, 15))
uncle_bob.save() # bob is now stored in the database
# Returns: 1

1

In [7]:
grandma = Person.create(name='Grandma', birthday=date(1935, 3, 1))
herb = Person.create(name='Herb', birthday=date(1950, 5, 5))

In [8]:
grandma.name = 'Grandma L.'
grandma.save()  # Update grandma's name in the database.
# Returns: 1

1

In [9]:
bob_kitty = Pet.create(owner=uncle_bob, name='Kitty', animal_type='cat')
herb_fido = Pet.create(owner=herb, name='Fido', animal_type='dog')
herb_mittens = Pet.create(owner=herb, name='Mittens', animal_type='cat')
herb_mittens_jr = Pet.create(owner=herb, name='Mittens Jr', animal_type='cat')

In [10]:
herb_mittens.delete_instance() # he had a great life
# Returns: 1

1

In [11]:
herb_fido.owner = uncle_bob
herb_fido.save()

1

In [12]:
grandma = Person.select().where(Person.name == 'Grandma L.').get()

In [14]:
grandma.name, grandma.birthday

('Grandma L.', datetime.date(1935, 3, 1))

In [15]:
grandma = Person.get(Person.name == 'Grandma L.')

In [18]:
grandma.name, grandma.birthday.strftime("%Y-%m-%d")

('Grandma L.', '1935-03-01')

In [19]:
for person in Person.select():
    print(person.name)

Bob
Grandma L.
Herb


In [20]:
query = Pet.select().where(Pet.animal_type == 'cat')
for pet in query:
    print(pet.name, pet.owner.name)

Kitty Bob
Mittens Jr Herb


In [21]:
query = (Pet
         .select(Pet, Person)
         .join(Person)
         .where(Pet.animal_type == 'cat'))

for pet in query:
    print(pet.name, pet.owner.name)

Kitty Bob
Mittens Jr Herb


In [22]:
for pet in Pet.select().join(Person).where(Person.name == 'Bob'):
    print(pet.name)

Kitty
Fido


In [23]:
for pet in Pet.select().where(Pet.owner == uncle_bob):
    print(pet.name)

Kitty
Fido


In [24]:
for pet in Pet.select().where(Pet.owner == uncle_bob).order_by(Pet.name):
    print(pet.name)

# prints:
# Fido
# Kitty

Fido
Kitty


In [25]:
for person in Person.select().order_by(Person.birthday.desc()):
    print(person.name, person.birthday)

# prints:
# Bob 1960-01-15
# Herb 1950-05-05
# Grandma L. 1935-03-01

Bob 1960-01-15
Herb 1950-05-05
Grandma L. 1935-03-01


In [26]:
d1940 = date(1940, 1, 1)
d1960 = date(1960, 1, 1)
query = (Person
         .select()
         .where((Person.birthday < d1940) | (Person.birthday > d1960)))

for person in query:
    print(person.name, person.birthday)

# prints:
# Bob 1960-01-15
# Grandma L. 1935-03-01

Bob 1960-01-15
Grandma L. 1935-03-01


In [27]:
query = (Person
         .select()
         .where(Person.birthday.between(d1940, d1960)))

for person in query:
    print(person.name, person.birthday)

# prints:
# Herb 1950-05-05

Herb 1950-05-05


In [28]:
for person in Person.select():
    print(person.name, person.pets.count(), 'pets')

# prints:
# Bob 2 pets
# Grandma L. 0 pets
# Herb 1 pets

Bob 2 pets
Grandma L. 0 pets
Herb 1 pets


In [29]:
query = (Person
         .select(Person, fn.COUNT(Pet.id).alias('pet_count'))
         .join(Pet, JOIN.LEFT_OUTER)  # include people without pets.
         .group_by(Person)
         .order_by(Person.name))


In [30]:
query

In [31]:
for person in query:
    # "pet_count" becomes an attribute on the returned model instances.
    print(person.name, person.pet_count, 'pets')

Bob 2 pets
Grandma L. 0 pets
Herb 1 pets


In [32]:
query = (Person
         .select(Person, Pet)
         .join(Pet, JOIN.LEFT_OUTER)
         .order_by(Person.name, Pet.name))
for person in query:
    # We need to check if they have a pet instance attached, since not all
    # people have pets.
    if hasattr(person, 'pet'):
        print(person.name, person.pet.name)
    else:
        print(person.name, 'no pets')

# prints:
# Bob Fido
# Bob Kitty
# Grandma L. no pets
# Herb Mittens Jr

Bob Fido
Bob Kitty
Grandma L. no pets
Herb Mittens Jr


In [1]:
print(12345)

12345
